## Problema del Viajante de Comercio con Ventanas de Tiempo (TSPTW)

El **Problema del Viajante de Comercio con Ventanas de Tiempo** (TSPTW) es una variante del clásico problema del Viajante de Comercio (TSP), donde, además de minimizar la distancia total recorrida, se deben respetar las restricciones de tiempo en cada uno de los nodos (localizaciones). En este problema, cada nodo tiene una ventana de tiempo en la que el viajante debe llegar, y no puede llegar antes ni después de esa ventana, aunque se permite llegar en cualquier momento dentro de la ventana.

### Enunciado del problema

Dado un conjunto de ciudades (localizaciones) y las distancias (o tiempos) entre cada par de ciudades, el objetivo es encontrar la ruta más corta que pase por todas las ciudades exactamente una vez, respetando las ventanas de tiempo de cada ciudad. Las ventanas de tiempo definen un intervalo en el que se puede llegar a cada ciudad, y el viajante no puede llegar fuera de este intervalo.

### Variables de decisión:

- $x_{ij}$: Variable binaria que indica si el viajante va de la ciudad \( i \) a la ciudad \( j \).
- $u_{i}$: Variable que representa la posición del viajante en la ruta, es decir, el orden en el que visita la ciudad \( i \).

### Parámetros:

- $d_{ij}$: Distancia (o tiempo) entre la ciudad \( i \) y la ciudad \( j \).
- $a_{i}$: Tiempo de inicio de la ventana de tiempo en la ciudad \( i \).
- $b_{i}$: Tiempo de fin de la ventana de tiempo en la ciudad \( i \).
- $t_{i}$: Tiempo de llegada a la ciudad \( i \).

### Función objetivo:

Minimizar la distancia total recorrida, es decir, minimizar la suma de las distancias entre las ciudades visitadas:

$$
\text{Minimizar} \quad \sum_{i,j \in C} d_{ij} \cdot x_{ij}
$$

donde \( C \) es el conjunto de todas las ciudades.

### Restricciones:

1. **Visitar todas las ciudades exactamente una vez**:
   
   Para cada ciudad \( i \), debe existir exactamente un \( j \) tal que el viajante vaya de \( i \) a \( j \), y viceversa:

   $$ 
   \sum_{j \in C, j \neq i} x_{ij} = 1 \quad \forall i \in C
   $$

   $$ 
   \sum_{i \in C, i \neq j} x_{ij} = 1 \quad \forall j \in C
   $$

2. **Ventanas de tiempo**:

   El tiempo de llegada a cada ciudad \( i \) debe estar dentro de la ventana de tiempo definida por $a_{i}$ y $b_{i}$:

   $$
   a_i \leq t_i \leq b_i \quad \forall i \in C
   $$

3. **Orden de visita**:

   Las variables de posición $u_{i}$ deben cumplir con las siguientes restricciones de orden:

   $$
   u_i - u_j + (n-1) \cdot x_{ij} \leq n-2 \quad \forall i,j \in C, i \neq j
   $$

   donde \( n \) es el número total de ciudades.

4. **Restricción de flujo**:

   Para cada ciudad \( i \), la posición $u_{i}$ debe ser mayor que la posición de la ciudad anterior en la ruta, y debe ser coherente con las variables de las aristas:

   $$ 
   t_i = t_{i-1} + d_{i,i-1} \quad \forall i
   $$

### Resolución con JuMP en Julia

A continuación, se presenta la resolución de este problema utilizando el paquete **JuMP** en Julia y el optimizador **GLPK**. En este caso, generamos las distancias y las ventanas de tiempo de manera aleatoria para crear un ejemplo que cumpla con las restricciones.


In [ ]:
using JuMP, GLPK
using Random
using LinearAlgebra

# Configuración inicial
n = 5  # Número de ciudades
Random.seed!(123)  # Semilla para reproducibilidad

# Generar coordenadas aleatorias para las ciudades
coord = rand(0:1000, n, 2)

# Generar costos entre ciudades
cost = zeros(Int, n, n)
for i in 1:n, j in 1:n
    cost[i, j] = round(sqrt((coord[i, 1] - coord[j, 1])^2 + (coord[i, 2] - coord[j, 2])^2), digits=0)
end

# Generar ventanas de tiempo ampliadas
ventanas_inicio = zeros(Int, n)
ventanas_fin = zeros(Int, n)
for i in 1:n
    ventanas_inicio[i] = (i - 1) * 50 + 10
    ventanas_fin[i] = ventanas_inicio[i] + 200  # Ampliar duración de la ventana
end

# Mostrar las ventanas generadas
println("Ventanas de tiempo:")
for i in 1:n
    println("Ciudad $i: [", ventanas_inicio[i], ", ", ventanas_fin[i], "]")
end

# Crear el modelo
model = Model(GLPK.Optimizer)

# Variables de decisión
@variable(model, x[1:n, 1:n], Bin)  # Variable binaria para cada par de ciudades
@variable(model, u[1:n] >= 0, Int)  # Variables de posición en la ruta

# Función objetivo: minimizar la distancia total recorrida
@objective(model, Min, sum(cost[i, j] * x[i, j] for i in 1:n, j in 1:n if i != j))

# Restricciones de flujo
@constraint(model, [i=1:n], sum(x[i, j] for j in 1:n if i != j) == 1)  # Cada ciudad debe ser visitada exactamente una vez
@constraint(model, [j=1:n], sum(x[i, j] for i in 1:n if i != j) == 1)  # Cada ciudad debe ser visitada exactamente una vez

# Restricciones de ventanas de tiempo
@constraint(model, [i=1:n], ventanas_inicio[i] <= u[i] <= ventanas_fin[i])  # Respetar ventanas de tiempo

# Restricciones de subciclo (MTZ)
@constraint(model, [i=2:n, j=2:n; i != j], u[i] - u[j] + n * x[i, j] <= n - 1)

# Resolver el modelo
optimize!(model)

# Mostrar los resultados
if termination_status(model) == MOI.OPTIMAL
    println("\nSolución óptima encontrada:")
    println("Distancia total: ", objective_value(model))
    println("Ruta:")
    for i in 1:n
        for j in 1:n
            if value(x[i, j]) > 0.5  # Interpretar la variable binaria
                println("Ciudad $i -> Ciudad $j")
            end
        end
    end
    println("\nPosiciones en la ruta:")
    for i in 1:n
        println("Ciudad $i: posición ", value(u[i]))
    end
else
    println("No se encontró una solución óptima.")
end

Ventanas de tiempo:
Ciudad 1: [10, 210]
Ciudad 2: [60, 260]
Ciudad 3: [110, 310]
Ciudad 4: [160, 360]
Ciudad 5: [210, 410]

Solución óptima encontrada:
Distancia total: 2457.0
Ruta:
Ciudad 1 -> Ciudad 5
Ciudad 2 -> Ciudad 3
Ciudad 3 -> Ciudad 4
Ciudad 4 -> Ciudad 1
Ciudad 5 -> Ciudad 2

Posiciones en la ruta:
Ciudad 1: posición 10.0
Ciudad 2: posición 211.0
Ciudad 3: posición 212.0
Ciudad 4: posición 214.0
Ciudad 5: posición 210.0
